In [504]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib

In [449]:
# %pip install faker

In [450]:
df = pd.read_excel('data/arquivo_anonimizado.xlsx')
df.drop(['Unnamed: 0', 'reprovacoes'], axis=1, inplace=True)

In [451]:
faker = Faker('pt_BR')

quantidade_de_nomes = df.shape[0]
nomes_completos = [faker.name() for _ in range(quantidade_de_nomes)]

df['nome_completo'] = nomes_completos

In [452]:
df.head(5).T

,0,1,2,3,4
matriculaDRE,0,1,2,3,4
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por abandono,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 062...,2000/1 - MAA123 Algebra para Informatica - 009...,2000/1 - MAA123 Algebra para Informatica - 040...,2000/1 - MAA123 Algebra para Informatica - 081...
crPorPeriodo,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 6.5\n2001/1 - 5.1\n2001...,2000/1 - 3.3\n2000/2 - 4.7\n2001/1 - 0.7,2000/1 - 6.3\n2000/2 - 5.1\n2001/1 - 2.8\n2001...,2000/1 - 8.3\n2000/2 - 7\n2001/1 - 7.2\n2001/2...
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7\n2001/1 - 6.4\n2001/2...,2000/1 - 3.3\n2000/2 - 3.9\n2001/1 - 3.2,2000/1 - 6.3\n2000/2 - 5.7\n2001/1 - 4.7\n2001...,2000/1 - 8.4\n2000/2 - 7.7\n2001/1 - 7.6\n2001...


In [453]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   matriculaDRE               2817 non-null   int64  
 1   periodoIngressoUFRJ        2817 non-null   object 
 2   cursoIngressoUFRJ          2817 non-null   object 
 3   cursoAtual                 2817 non-null   object 
 4   periodoIngressoCursoAtual  2817 non-null   object 
 5   situacaoMatriculaAtual     2817 non-null   object 
 6   sexo                       2817 non-null   object 
 7   disciplinasCursadas        2650 non-null   object 
 8   crPorPeriodo               2644 non-null   object 
 9   craPorPeriodo              2642 non-null   object 
 10  periodosIntegralizados     2817 non-null   int64  
 11  cargaHorariaAcumulada      2817 non-null   int64  
 12  CRA                        2654 non-null   float64
 13  periodosTrancados          948 non-null    objec

# Processamento

Removendo alunos que não possui disciplinas cursas, pois se não há disciplinas, não há metricas a serem analisadas.

In [454]:
df.drop(['CRA', 'periodosCRMenor3', 'periodosTrancados', 'periodosCancelados'], axis=1, inplace=True)
df.dropna(subset=['disciplinasCursadas'], inplace=True)

## CR por Periodo

In [455]:
df['crPorPeriodo'] = df['crPorPeriodo'].astype(str)
df['crPorPeriodo'] = df['crPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('crPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.1)","(2001/1, 5.9)","(2001/2, 6.8)","(2002/1, 6.3)"
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...


In [456]:
df_crPeriodo = pd.DataFrame(df_exploded['crPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cr'))
df_crPeriodo

,periodo,cr
0,2000/1,7.6
0,2000/2,7.1
0,2001/1,5.9
0,2001/2,6.8
0,2002/1,6.3
...,...,...
2775,2022/1,7.5
2776,2022/1,7.8
2777,2022/1,7.9
2778,2022/1,6.7


In [457]:
df_fato = pd.concat([df_exploded, df_crPeriodo], axis=1)
df_fato.drop('crPorPeriodo', axis=1, inplace=True)
df_fato.sample(5).T

,1041,1077,2239,1886,837
matriculaDRE,1041,1077,2239,1886,837
periodoIngressoUFRJ,2008/1,2008/1,2017/2,2014/2,2006/1
cursoIngressoUFRJ,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2008/1,2008/1,2021/1,2014/2,2006/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por abandono Definitivo,Ativa,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2008/1 - MAB111 Fund da Computação Digital - 1...,2008/1 - MAB111 Fund da Computação Digital - 0...,2017/0 - MAB115 Álgebra Linear Algorítmica - T...,2014/2 - MAB111 Fund da Computação Digital - 0...,2006/1 - MAB111 Fund da Computação Digital - 0...
craPorPeriodo,2008/1 - 5.3\n2008/2 - 6\n2009/1 - 6.3\n2009/2...,2008/1 - 7.4\n2008/2 - 6.3\n2009/1 - 5.8\n2009...,2017/2 - 6.5\n2018/1 - 7.1\n2018/2 - 6.9\n2019...,2014/2 - 8.3\n2015/1 - 8.4\n2015/2 - 8.2\n2016...,2006/1 - 8\n2006/2 - 8.1\n2007/1 - 8.1\n2007/2...
periodosIntegralizados,15,6,3,10,9


## CRA por Periodo

In [458]:
df['craPorPeriodo'] = df['craPorPeriodo'].astype(str)
df['craPorPeriodo'] = df['craPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('craPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.4)","(2001/1, 6.9)","(2001/2, 6.9)","(2002/1, 6.8)"


In [459]:
df_craPeriodo = pd.DataFrame(df_exploded['craPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cra'))
df_craPeriodo['matriculaDRE'] = df_exploded['matriculaDRE']
df_craPeriodo

,periodo,cra,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.4,0
0,2001/1,6.9,0
0,2001/2,6.9,0
0,2002/1,6.8,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


In [460]:
df_fato.drop('craPorPeriodo', axis=1, inplace=True)
df_fato = pd.merge(df_fato, df_craPeriodo, on=['periodo', 'matriculaDRE'], how='left')
df_fato.head(5).T

,0,1,2,3,4
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
periodosIntegralizados,13,13,13,13,13
cargaHorariaAcumulada,3105,3105,3105,3105,3105


## Disciplinas Cursadas

Nessa seção vamos separar a lista de disciplinas em varias colunas, uma para cada disciplina

In [461]:
df['disciplinasCursadas'] = df['disciplinasCursadas'].astype(str)
df['disciplinasCursadas'] = df['disciplinasCursadas'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])

In [462]:
df_exploded = df.explode('disciplinasCursadas')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,"(2000/1, MAA123 Algebra para Informatica, 060,...","(2000/1, MAB111 Fund da Computação Digital, 08...","(2000/1, MAB120 Computacao para Informatica, 0...","(2000/1, MAE111 Cálculo Infinitesimal I, 056, ...","(2000/1, MAE115 Cálculo Vetorial e G Analitica..."
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ..."


Agora vamos pegar as informações de cada desciplina e desmembrar em novas colunas

In [463]:
df_exploded['disciplinasCursadas'] = df_exploded['disciplinasCursadas'].apply(lambda x: (
    x[0].replace(' ', ''),
    x[1].split()[0].replace(' ', ''),
    ' '.join(x[1].split()[1:]),
    x[2].replace(' ', ''),
    x[3].replace(' ', '')
))

In [464]:
df_disciplina = pd.DataFrame(df_exploded['disciplinasCursadas'].tolist(), index=df_exploded.index, columns=('periodo','cod_Disciplina','disciplina','grau_disciplina','situacao_disciplina'))
df_disciplina['matriculaDRE'] = df_exploded['matriculaDRE']
df_disciplina

,periodo,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0
0,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0
0,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0
0,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0
0,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0
...,...,...,...,...,...,...
2779,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779
2779,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779
2779,2022/1,ICP120,Computação I (CC),075,Aprovado,2779
2779,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779


In [465]:
df_fato.drop('disciplinasCursadas',axis=1, inplace=True)
df_fato = pd.merge(df_fato, df_disciplina, on=['periodo', 'matriculaDRE'], how='left')
df_fato.head(5).T

,0,1,2,3,4
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
periodosIntegralizados,13,13,13,13,13
cargaHorariaAcumulada,3105,3105,3105,3105,3105
nome_completo,Marcela da Luz,Marcela da Luz,Marcela da Luz,Marcela da Luz,Marcela da Luz


In [469]:
df_fato[['ano', 'semestre']] = df_fato['periodo'].str.split('/', expand=True)
df_fato.head(5).T

,0,1,2,3,4
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
periodosIntegralizados,13,13,13,13,13
cargaHorariaAcumulada,3105,3105,3105,3105,3105
nome_completo,Marcela da Luz,Marcela da Luz,Marcela da Luz,Marcela da Luz,Marcela da Luz


# Dimenssão Aluno

In [524]:
colunas_aluno = ['sk_d_aluno', 'nome_completo', 'matriculaDRE', 'sexo']

In [525]:
d_aluno = df_fato[colunas_aluno[1:]]

# d_aluno.groupby(['nome_completo', 'matriculaDRE', 'sexo']).first().reset_index()
d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)
d_aluno

,nome_completo,matriculaDRE,sexo
0,Marcela da Luz,0,M
1,Larissa Nogueira,1,M
2,Pietro Fogaça,2,M
3,Bruno Silveira,3,M
4,Gabriela Santos,4,M
...,...,...,...
2645,Cauã Monteiro,2775,M
2646,Theo Gonçalves,2776,M
2647,Nicole Viana,2777,M
2648,Amanda Oliveira,2778,M


In [529]:
d_aluno['sk_d_aluno'] = d_aluno.apply(lambda x: hashlib.sha256(str(x['matriculaDRE']).encode()).hexdigest(), axis=1)
d_aluno = d_aluno.reindex(columns=colunas_aluno)
d_aluno.to_csv('data/d_aluno.csv', index=False)
d_aluno

,sk_d_aluno,nome_completo,matriculaDRE,sexo
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,Marcela da Luz,0,M
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Larissa Nogueira,1,M
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Pietro Fogaça,2,M
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Bruno Silveira,3,M
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Gabriela Santos,4,M
...,...,...,...,...
2645,3da16474e33b330b71e5b562cfcf7939bde3c12a08b746...,Cauã Monteiro,2775,M
2646,9b3cea3bd8789ba47cc969bff5914da7ebac8e0052e282...,Theo Gonçalves,2776,M
2647,5a48eed290f62c93553855c36c964e1ef16603d23dcce3...,Nicole Viana,2777,M
2648,a2541e84e912f4f5dd32ce131abdb6b1fdc9cf315790f1...,Amanda Oliveira,2778,M


# Dimenssão Disciplina

In [537]:
# def criar_dimenssao(nome_dimenssao, colunas_dimenssao, hash):
#     dimenssao = df_fato[colunas_dimenssao[1:]]
#     dimenssao = dimenssao.drop_duplicates().reset_index(drop=True)
#     dimenssao[f'sk_d_{nome_dimenssao}'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x[hash]).encode()).hexdigest(), axis=1)
#     d_disciplina = d_disciplina.reindex(columns=colunas_aluno)
#     d_disciplina.to_csv(f'data/d_{nome_dimenssao}.csv', index=False)
#     d_disciplina

In [538]:
colunas_disciplina = ['sk_d_disciplina', 'cod_Disciplina', 'disciplina']

In [541]:
d_disciplina = df_fato[colunas_disciplina[1:]]
d_disciplina = d_disciplina.drop_duplicates().reset_index(drop=True)
d_disciplina

,cod_Disciplina,disciplina
0,MAA123,Algebra para Informatica
1,MAB111,Fund da Computação Digital
2,MAB120,Computacao para Informatica
3,MAE111,Cálculo Infinitesimal I
4,MAE115,Cálculo Vetorial e G Analitica
...,...,...
1517,ICP121,Computação I
1518,FIW483,História da Física
1519,NEP144,Tóp Esp Polít Públ Dir Hum VI
1520,NEP142,Tóp Esp Polít Públ Dir Hum IV


In [542]:
d_disciplina['sk_d_disciplina'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x['cod_Disciplina']).encode()).hexdigest(), axis=1)
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)
d_disciplina.to_csv('data/d_disciplina.csv', index=False)
d_disciplina

,sk_d_disciplina,cod_Disciplina,disciplina
0,3f49725780c6186ef08c8772a9805d5ba45a818b2c2ac8...,MAA123,Algebra para Informatica
1,e5ff4465a19412b184ede199b4a8b480c7b130890a02ae...,MAB111,Fund da Computação Digital
2,8c0e7f3209f34c0e3dbd9901137fda665888f0abb015bd...,MAB120,Computacao para Informatica
3,4e39ab690a3348811d61e82524b1057ef593a531f7e18f...,MAE111,Cálculo Infinitesimal I
4,1aff777b25a29471b147e5493f3d0e0b8c6501f70f2ebc...,MAE115,Cálculo Vetorial e G Analitica
...,...,...,...
1517,0bd1ac82671ad0e3594a478cc27fec73cff8397330090d...,ICP121,Computação I
1518,9765a9d39b0846d92607c79c4da55bcfdbef44f42910c7...,FIW483,História da Física
1519,2f1f5279ef45edb6784daef6d4a582da86c66249f7e6e3...,NEP144,Tóp Esp Polít Públ Dir Hum VI
1520,fb4bbfc6cec8c847877fd1ed7249024fc6403b2bfe708d...,NEP142,Tóp Esp Polít Públ Dir Hum IV


# Dimenssão Periodo

In [553]:
colunas_periodo = ['sk_d_periodo', 'ano', 'semestre']

In [555]:
d_periodo = df_fato[colunas_periodo[1:]]
d_periodo = d_periodo.drop_duplicates().reset_index(drop=True)
d_periodo

,ano,semestre
0,2000,1
1,2000,2
2,2001,1
3,2001,2
4,2002,1
5,2002,2
6,2003,1
7,2003,2
8,2004,1
9,2004,2
